In [20]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [6]:
data = pd.read_csv("lending_club_cleaned_v1.csv")
data.head()

,term,fico_range_low,total_acc,pub_rec,revol_util,annual_inc,int_rate,dti,mort_acc,loan_amnt,...,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,application_type_Joint App,initial_list_status_w
0,60,660.0,18.0,0.0,86.8,40440.0,16.99,15.16,1.0,17775.0,...,0,0,0,0,0,0,0,1,0,0
1,60,740.0,26.0,0.0,103.5,59000.0,15.61,16.74,4.0,29175.0,...,0,0,0,0,0,0,0,1,0,0
2,36,700.0,47.0,0.0,11.4,40000.0,7.90,20.34,1.0,6000.0,...,0,0,0,0,0,1,0,0,0,1
3,60,665.0,26.0,0.0,56.2,70000.0,16.99,23.15,7.0,15600.0,...,0,0,0,0,0,0,0,1,0,0
4,36,700.0,15.0,1.0,67.1,120000.0,14.98,17.88,1.0,10000.0,...,0,0,1,0,0,0,0,1,0,0


In [7]:
x = data.drop(columns=['loan_status'])
y = data['loan_status']

In [8]:
scaling = StandardScaler()

In [9]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [10]:
x_train_scaled = scaling.fit_transform(x_train)
x_test_scaled = scaling.fit_transform(x_test)

In [14]:
clf = DecisionTreeClassifier(random_state=42)

In [12]:
print(clf.get_params().keys())

dict_keys(['ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'monotonic_cst', 'random_state', 'splitter'])


In [15]:
param_grid = {
    'criterion' : ['entropy' , 'gini'],
    'max_depth' : [0,5,10,20],
    'min_samples_leaf' : range(2,100,10),
    'min_samples_split' : range(1,50,5),
    'class_weight':[None,'balanced']
}
random_search = RandomizedSearchCV(
    estimator=clf,
    param_distributions=param_grid,
    n_iter=50,
    cv=5,
    scoring='f1',
    random_state=42,
    verbose=2
)
random_search.fit(x_train_scaled,y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END class_weight=None, criterion=gini, max_depth=5, min_samples_leaf=22, min_samples_split=31; total time=   1.2s
[CV] END class_weight=None, criterion=gini, max_depth=5, min_samples_leaf=22, min_samples_split=31; total time=   1.1s
[CV] END class_weight=None, criterion=gini, max_depth=5, min_samples_leaf=22, min_samples_split=31; total time=   1.2s
[CV] END class_weight=None, criterion=gini, max_depth=5, min_samples_leaf=22, min_samples_split=31; total time=   1.3s
[CV] END class_weight=None, criterion=gini, max_depth=5, min_samples_leaf=22, min_samples_split=31; total time=   1.2s
[CV] END class_weight=None, criterion=entropy, max_depth=20, min_samples_leaf=52, min_samples_split=21; total time=   3.4s
[CV] END class_weight=None, criterion=entropy, max_depth=20, min_samples_leaf=52, min_samples_split=21; total time=   3.1s
[CV] END class_weight=None, criterion=entropy, max_depth=20, min_samples_leaf=52, min_samples_spl

e:\python\Lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
65 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
55 fits failed with the following error:
Traceback (most recent call last):
  File "e:\python\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "e:\python\Lib\site-packages\sklearn\base.py", line 1358, in wrapper
    estimator._validate_params()
  File "e:\python\Lib\site-packages\sklearn\base.py", line 471, in _validate_params
    validate_parameter_constraints(
  File "e:\python\Lib\site-packages\sklearn\utils\_param_validation.py", line 98, in validate_parameter_constr

,estimator,DecisionTreeC...ndom_state=42)
,param_distributions,"{'class_weight': [None, 'balanced'], 'criterion': ['entropy', 'gini'], 'max_depth': [0, 5, ...], 'min_samples_leaf': range(2, 100, 10), ...}"
,n_iter,50
,scoring,'f1'
,n_jobs,None
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


In [16]:
results = pd.DataFrame(random_search.cv_results_)
results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_min_samples_split,param_min_samples_leaf,param_max_depth,param_criterion,param_class_weight,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1.290653,0.062788,0.015690,0.003054,31,22,5,gini,None,"{'min_samples_split': 31, 'min_samples_leaf': ...",0.911184,0.912066,0.911869,0.909644,0.912409,0.911434,0.000981,2
1,3.252707,0.174853,0.024686,0.004276,21,52,20,entropy,None,"{'min_samples_split': 21, 'min_samples_leaf': ...",0.908942,0.908794,0.909146,0.907864,0.909333,0.908816,0.000509,14
2,1.322897,0.061335,0.014094,0.001957,41,62,5,entropy,None,"{'min_samples_split': 41, 'min_samples_leaf': ...",0.909978,0.909603,0.910769,0.909512,0.910682,0.910109,0.000528,10
3,1.328756,0.025627,0.007116,0.006327,26,32,5,entropy,None,"{'min_samples_split': 26, 'min_samples_leaf': ...",0.909978,0.909603,0.910769,0.909512,0.910682,0.910109,0.000528,10
4,1.434227,0.038331,0.011074,0.005999,36,32,5,entropy,balanced,"{'min_samples_split': 36, 'min_samples_leaf': ...",0.829798,0.826505,0.815062,0.830063,0.832436,0.826773,0.006153,23


In [17]:
best_parameter = random_search.best_params_

In [19]:
best_clf = random_search.best_estimator_
y_predict = best_clf.predict(x_test_scaled)

In [21]:
print(classification_report(y_test,y_predict))

              precision    recall  f1-score   support

           0       0.58      0.25      0.35      7951
           1       0.86      0.96      0.91     39419

    accuracy                           0.84     47370
   macro avg       0.72      0.61      0.63     47370
weighted avg       0.82      0.84      0.82     47370



In [22]:
print(confusion_matrix(y_test,y_predict))

[[ 2010  5941]
 [ 1432 37987]]


Here the above one fail with the recall for 0 is 0.25 which is not good

## Now creating a new one with high recall score for 0(defaulters)

changing the using data from scaled data to default traning data without scaled and maily focusing it to catch the defaulter so that scoring is for 0

In [24]:
from sklearn.metrics import make_scorer,recall_score

In [25]:
recall_0 = make_scorer(recall_score,pos_label=0)

In [31]:
param_grid_2 = {
    'criterion' : ['entropy' , 'gini'],
    'max_depth' : [3,5,10,20],
    'min_samples_leaf' : range(5,100,10),
    'min_samples_split' : range(5,200,20),
    'class_weight':['balanced']
}
random_search_2 = RandomizedSearchCV(
    estimator=clf,
    param_distributions=param_grid,
    n_iter=30,
    cv=5,
    scoring=recall_0,
    random_state=42,
    verbose=1
)
random_search_2.fit(x_train,y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


,estimator,DecisionTreeC...ndom_state=42)
,param_distributions,"{'class_weight': ['balanced'], 'criterion': ['entropy', 'gini'], 'max_depth': [3, 5, ...], 'min_samples_leaf': range(5, 100, 10), ...}"
,n_iter,30
,scoring,"make_scorer(r..., pos_label=0)"
,n_jobs,None
,refit,True
,cv,5
,verbose,1
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


In [32]:
best_clf_2 = random_search_2.best_estimator_
y_predict_2 = best_clf_2.predict(x_test)

In [33]:
print(classification_report(y_test,y_predict_2))

              precision    recall  f1-score   support

           0       0.31      0.76      0.44      7951
           1       0.93      0.65      0.77     39419

    accuracy                           0.67     47370
   macro avg       0.62      0.71      0.60     47370
weighted avg       0.83      0.67      0.71     47370



In [34]:
import joblib

joblib.dump(random_search_2.best_estimator_, 'loan_decision_tree.pkl')


['loan_decision_tree.pkl']